In [42]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch

# processor = AutoImageProcessor.from_pretrained("al-css/Screenshots_detection_to_classification")
model = AutoModelForImageClassification.from_pretrained("al-css/Screenshots_detection_to_classification")
processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

/opt/conda/envs/fusemix/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [30]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("fxmarty/clip-vision-model-tiny")

In [ ]:
from io import BytesIO
from PIL import Image
import requests

model.eval()
model.to(torch.float16)
image_path = '/workspace/repo/llm2clip/EVA-CLIP/tmp/bing_images/0_4_7.jpeg'
# convert to tensor
image = Image.open(image_path)
# image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, return_tensors="pt", padding=True)
# PIL.Image.Image
outputs = model(**inputs)


Unused or unrecognized kwargs: padding.


In [33]:
model

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 128, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (position_embedding): Embedding(50, 128)
    )
    (pre_layrnorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-1): 2 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=128, out_features=128, bias=True)
            (v_proj): Linear(in_features=128, out_features=128, bias=True)
            (q_proj): Linear(in_features=128, out_features=128, bias=True)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
          (layer_norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=128, out_features=3072, bias=True)
          

In [35]:
outputs[-1].shape

torch.Size([1, 128])

In [25]:
outputs

ImageClassifierOutput(loss=None, logits=tensor([[-4.0352,  4.3125]], dtype=torch.float16, grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [1]:
from transformers import CLIPProcessor, CLIPModel, CLIPConfig
from PIL import Image
import requests
import torch

model_id = "yujiepan/clip-vit-tiny-random-patch14-336"
model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

/opt/conda/envs/fusemix/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/fusemix/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [41]:
import os

image_roots = '/workspace/repo/llm2clip/EVA-CLIP/tmp/bing_images/TeamViewer/TeamViewer-session-list'

image_paths = os.listdir(image_roots)

with torch.no_grad():
	for image_path in image_paths:
		image = Image.open(os.path.join(image_roots, image_path))
		texts = ["Screenshot", "photograph"," Artwork","Graphic"]
		inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
		outputs = model(**inputs)
		logits_per_image = outputs.logits_per_image
		probs = logits_per_image
		result_index = torch.argmax(probs)
		print(f"Image: {image_path}, Prediction: {texts[result_index]}, Probability: {probs}")
		

Image: 0_4_77.jpeg, Prediction: Graphic, Probability: tensor([[ 4.1386,  1.4008, -0.2838,  4.6064]])
Image: 0_4_86.jpeg, Prediction: Screenshot, Probability: tensor([[ 2.3969, -1.7005, -2.6309,  1.5089]])
Image: 0_4_4.jpeg, Prediction: Screenshot, Probability: tensor([[ 1.1040, -1.6138, -2.5328,  0.3238]])
Image: 0_4_15.jpeg, Prediction: Graphic, Probability: tensor([[ 3.8941,  1.2394, -0.3086,  4.3425]])
Image: 0_4_73.jpeg, Prediction: Graphic, Probability: tensor([[ 4.1113,  1.6057, -0.0648,  4.6256]])
Image: 0_4_63.jpeg, Prediction: Graphic, Probability: tensor([[ 4.1580,  1.5433, -0.0778,  4.6855]])
Image: 0_4_85.jpeg, Prediction: Screenshot, Probability: tensor([[ 1.1240, -1.5760, -2.5020,  0.3503]])
Image: 0_4_5.jpeg, Prediction: Graphic, Probability: tensor([[ 3.9393,  1.3604, -0.2450,  4.3652]])
Image: 0_4_22.jpeg, Prediction: Graphic, Probability: tensor([[ 3.8994,  1.0342, -0.5647,  4.2048]])
Image: 0_4_68.jpeg, Prediction: Graphic, Probability: tensor([[4.2290, 1.7465, 0.109

In [47]:
import os

image_roots = '/workspace/repo/llm2clip/EVA-CLIP/tmp/bing_images/TeamViewer/TeamViewer-session-list'

image_paths = os.listdir(image_roots)

with torch.no_grad():
	for image_path in image_paths:
		image = Image.open(os.path.join(image_roots, image_path))
		inputs = processor(images=image, return_tensors="pt")
		outputs = model(**inputs)
		probs = outputs[-1]
		flag = probs.argmax().item()
		if flag == 0:
			print(f"Image: {image_path}, flag:{flag}, Probability: {probs}")

Image: 0_4_4.jpeg, flag:0, Probability: tensor([[ 1.2763, -1.6131]])
Image: 0_4_85.jpeg, flag:0, Probability: tensor([[ 1.2627, -1.6765]])
Image: 0_4_68.jpeg, flag:0, Probability: tensor([[ 0.0630, -0.6162]])
Image: 0_4_9.jpeg, flag:0, Probability: tensor([[ 2.3956, -2.9959]])
Image: 0_4_69.jpeg, flag:0, Probability: tensor([[ 1.7534, -0.6616]])
Image: 0_4_67.jpeg, flag:0, Probability: tensor([[ 1.7777, -1.6568]])
Image: 0_4_46.jpeg, flag:0, Probability: tensor([[ 1.2627, -1.6765]])
Image: 0_4_79.jpeg, flag:0, Probability: tensor([[ 1.7534, -0.6616]])
Image: 0_4_66.jpeg, flag:0, Probability: tensor([[ 2.6095, -1.6284]])
Image: 0_4_42.jpeg, flag:0, Probability: tensor([[ 0.0630, -0.6162]])
Image: 0_4_55.jpeg, flag:0, Probability: tensor([[ 1.7777, -1.6568]])
Image: 0_4_27.jpeg, flag:0, Probability: tensor([[ 2.6095, -1.6284]])
Image: 0_4_62.jpeg, flag:0, Probability: tensor([[ 0.6416, -0.6572]])


In [1]:
import os, sys
from PIL import Image

target_cwd = '/workspace/repo/cccrawl'
sys.path.append(target_cwd)

from bing_search.models import ScreenshotChecker

In [2]:
screenshot_checker = ScreenshotChecker()

In [3]:
import os

image_roots = '/workspace/repo/llm2clip/EVA-CLIP/tmp/bing_images_any/Excel-chart-tools'

image_paths = os.listdir(image_roots)

for image_path in image_paths:
	image = Image.open(os.path.join(image_roots, image_path))
	flag = screenshot_checker.predict(image)

In [12]:
float(flag[0].norm(1))

7.8107829093933105

In [27]:
from PIL import Image
import requests
from io import BytesIO

content_url = 'https://www.screenshotonpc.com/wp-content/uploads/2020/06/screenshot-word-click-screenshot-microsoft-3.png'

content_url = 'https://www.screenshotonpc.com/wp-content/uploads/2020/06/microsoft-word-featured-image-1.png'

content_url = "http://www.goskills.com/Video/1233/thumbnail-720p.jpg?t=T51444"


content_url = "https://www.researchgate.net/profile/Adam-Fourney-2/publication/316560440/figure/download/fig13/AS:488373489016840@1493448722310/Screenshot-of-the-Microsoft-Word-Online-user-interface-37-a-web-based.png"


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    # 'Accept-Language': 'en-US,en;q=0.9',
    # 'Accept-Encoding': 'gzip, deflate, br',
    # 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    # 'Connection': 'close',
    # 'Upgrade-Insecure-Requests': '1'
}

response = requests.get(content_url, headers=headers, stream=True) #发现出现很多
response.raise_for_status()
image = Image.open(BytesIO(response.content))

image

HTTPError: 403 Client Error: Forbidden for url: https://www.researchgate.net/profile/Adam-Fourney-2/publication/316560440/figure/download/fig13/AS:488373489016840@1493448722310/Screenshot-of-the-Microsoft-Word-Online-user-interface-37-a-web-based.png